## Read data function

In [1]:
import pandas as pd
import numpy as np

from dataclasses import dataclass
from typing import Any, List, Tuple, Dict
from pandas import DataFrame

@dataclass
class City:
    index : int
    X : float
    Y : float

@dataclass
class Item:
    index : int
    Profit : int
    Weight : int
    Node : int

@dataclass
class TTP:
    Name :str = None
    DTYPE : str = None
    Dimension : int = 0
    ITEMS : int = 0
    CAPACITY : int = 0
    MIN_SPEED : float = 0
    MAX_SPEED : float = 0
    RENTING_RATIO : float = 0
    EDGE_W : str = None
    NODE : List[City] = None
    ITEM : List[Item] = None

@dataclass
class TTPpd:
    Name :str = None
    DTYPE : str = None
    Dimension : int = 0
    ITEMS : int = 0
    CAPACITY : int = 0
    MIN_SPEED : float = 0
    MAX_SPEED : float = 0
    RENTING_RATIO : float = 0
    EDGE_W : str = None
    NODE : DataFrame = None
    ITEM : DataFrame = None

In [100]:
path = './datasets/test-example-n4.txt'
path2 = './datasets/a280-n1395.txt'
node , item = read_problem(file_path=path)
print(node,'\n', item)

     X    Y
0  0.0  0.0
1  4.0  0.0
2  8.0  3.0
3  0.0  3.0 
    Profit  Weight  Node
0      34      30     2
1      40      40     3
2      25      21     4


## Algorithm part

In [192]:
import pandas as pd
import random

max_weight = 80
max_speed = 1
min_speed = 0.1
renting_ratio = 1.516

def generate_ttp_solution(number_of_cities, items_df, knapsack_capacity):
    # Generate a random path (tour)
    path = np.random.permutation(number_of_cities) + 1

    # Initialize knapsack plan with no items picked
    plan = [0] * len(items_df)
    current_weight = 0

    # Randomly decide to pick up items considering the knapsack capacity
    for i, row in items_df.iterrows():
        item_weight = row['Weight']
        if current_weight + item_weight <= knapsack_capacity:
            decision = random.choice([0, 1])
            plan[i] = decision
            current_weight += item_weight * decision

    return path, plan

def euclidean_distance(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def calculate_time_and_profit(solution: List[int], plan: List[int], nodes_df, items_df, min_speed, max_speed, max_weight):
    total_time = 0
    total_profit = 0
    current_weight = 0

    # Calculate total profit from picked items
    for i, is_picked in enumerate(plan):
        if is_picked:
            total_profit += items_df.loc[i, 'Profit']

    # Calculate the total travel time
    for i in range(len(solution)):
        current_city = solution[i] - 1
        next_city = solution[0] - 1 if i == len(solution) - 1 else solution[i + 1] - 1

        # Update current weight based on items picked at the current city
        for j, is_picked in enumerate(plan):
            if is_picked and items_df.loc[j, 'Node'] == solution[i]:
                current_weight += items_df.loc[j, 'Weight']

        # Calculate speed based on current weight
        speed = max_speed - (current_weight / max_weight) * (max_speed - min_speed)
        speed = max(speed, min_speed)  # Ensure speed doesn't drop below minimum

        # Distance between current city and next city
        distance = euclidean_distance(
            (nodes_df.loc[current_city, 'X'], nodes_df.loc[current_city, 'Y']),
            (nodes_df.loc[next_city, 'X'], nodes_df.loc[next_city, 'Y'])
        )

        # Update time with time to next city
        total_time += distance / speed
 
    return total_time, total_profit

In [193]:
def calculate_time_and_profit(solution: List[int], plan: List[int], nodes_df, items_df, min_speed, max_speed, max_weight):
    total_time = 0
    total_profit = 0
    current_weight = 0

    # Calculate total profit from picked items
    for i, is_picked in enumerate(plan):
        if is_picked:
            total_profit += items_df.loc[i, 'Profit']

    # Calculate the total travel time
    for i in range(len(solution)):
        current_city_index = solution[i] - 1  # Adjust if solution uses 1-based indexing
        next_city_index = solution[0] - 1 if i == len(solution) - 1 else solution[i + 1] - 1

        # Ensure indices are within the valid range
        current_city_index = max(0, min(current_city_index, len(nodes_df) - 1))
        next_city_index = max(0, min(next_city_index, len(nodes_df) - 1))

        # Update current weight based on items picked at the current city
        for j, row in items_df.iterrows():
            if plan[j] == 1 and row['Node'] == solution[i]:
                current_weight += row['Weight']

        # Calculate speed based on current weight
        speed = max_speed - (current_weight / max_weight) * (max_speed - min_speed)
        speed = max(speed, min_speed)  # Ensure speed doesn't drop below minimum

        # Distance between current city and next city
        distance = euclidean_distance(
            (nodes_df.loc[current_city_index, 'X'], nodes_df.loc[current_city_index, 'Y']),
            (nodes_df.loc[next_city_index, 'X'], nodes_df.loc[next_city_index, 'Y'])
        )

        # Update time with time to next city
        total_time += distance / speed

    return total_time, total_profit


In [199]:
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def calculate_time_and_profit(solution, plan, nodes_df, items_df, min_speed, max_speed, max_weight):
    total_time = 0
    total_profit = 0
    current_weight = 0

    # Calculate total profit from picked items
    for i, is_picked in enumerate(plan):
        if is_picked:
            total_profit += items_df.iloc[i]['Profit']

    # Calculate the total travel time
    for i in range(len(solution)):
        current_city_index = solution[i] - 1
        next_city_index = solution[0] - 1 if i == len(solution) - 1 else solution[i + 1] - 1

        # Update current weight based on items picked at the current city
        for j, row in items_df.iterrows():
            if plan[j] == 1 and row['Node'] == solution[i]:
                current_weight += row['Weight']

        # Calculate speed based on current weight
        speed = max_speed - (current_weight / max_weight) * (max_speed - min_speed)
        speed = max(speed, min_speed)  # Ensure speed doesn't drop below minimum

        # Distance between current city and next city
        distance = euclidean_distance(
            (nodes_df.iloc[current_city_index]['X'], nodes_df.iloc[current_city_index]['Y']),
            (nodes_df.iloc[next_city_index]['X'], nodes_df.iloc[next_city_index]['Y'])
        )

        # Update time with time to next city
        total_time += distance / speed

    return total_time, total_profit


In [212]:
path , plan = generate_ttp_solution(4,item,80)
# len(path) , len(plan)
path , plan

(array([4, 2, 1, 3]), [1, 0, 1])

In [216]:
# path = [1,4,3,2]
# plan = [0,0,0]
path = [1,2,3,4]
plan = [0,1,1]

In [217]:
cost = calculate_time_and_profit(path,plan,node,item,min_speed,max_speed,max_weight)
cost

(33.107207533502354, 65)

In [231]:
# particle construction
min_speed = 0.1
max_speed = 1.0
max_weight=80

@dataclass
class Particle:
    path : List[int]
    plan : List[int]
    time : float = 0
    profit : float = 0

    def __post_init__(self):
        time , profit = calculate_time_and_profit(self.path,self.plan,node,item,min_speed,max_speed,max_weight)
        self.time = time
        self.profit = profit

In [232]:
p1 = Particle([1,4,3,2],[1,1,0])

In [233]:
p1

Particle(path=[1, 4, 3, 2], plan=[1, 1, 0], time=38.91443850267379, profit=74)

In [243]:
n = 3
[Particle(*generate_ttp_solution(4,item,80)) for _ in range(n)]

[Particle(path=array([2, 1, 3, 4]), plan=[0, 0, 0], time=25.544003745317532, profit=0),
 Particle(path=array([1, 2, 3, 4]), plan=[0, 0, 1], time=20.927986906710313, profit=25),
 Particle(path=array([3, 4, 2, 1]), plan=[1, 1, 0], time=82.66696949668142, profit=74)]

In [230]:
cost = calculate_time_and_profit([3,2,1,4],[1,0,1],node,item,min_speed,max_speed,max_weight)
cost

(34.334366181596856, 59)